In [ ]:
from pyspark.sql import Window
import pyspark.sql.functions as F
from main import process_imdb_data
from pyspark.sql import DataFrame
from pyspark.sql.functions import desc
import os


In [36]:
dataframes = process_imdb_data()
basics = dataframes["basics"]
akas = dataframes["akas"]
crew = dataframes["crew"]
ratings = dataframes["ratings"]
principals = dataframes["principals"]
name_df = dataframes["name"]
episode = dataframes["episode"]


RESULTS_DIR = "results"
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)


1. Які англомовні фільми мають рейтинг вище 7.0?

In [37]:

def english_high_rated_movies() -> DataFrame:
    print("\nАнгломовні фільми з рейтингом > 7.0:")
    result = basics.filter((F.col("titleType") == "movie") &
                           (F.col("startYear").isNotNull())) \
                   .join(ratings, "tconst", "inner") \
                   .filter(F.col("averageRating") > 7.0) \
                   .select("tconst", "primaryTitle", "startYear", "averageRating", "numVotes")
    result.show(truncate=False)
    print(f"Загальна кількість записів у відповіді: {result.count()}")
    result.coalesce(1).write.option("header", "true").csv(f"{RESULTS_DIR}/english_high_rated_movies.csv")
    return result



2. Які режисери працювали протягом кількох десятиліть з високими рейтингами, і в якому десятилітті кожен з них показав найкращий середній рейтинг?

In [38]:
def directors_best_decade() -> DataFrame:
    print("\nРежисери з найкращим десятиліттям за рейтингом:")
    movies_df = basics.filter((F.col("titleType") == "movie") & (F.col("startYear").isNotNull()))
    movies_with_ratings = movies_df.join(ratings, "tconst")
    movies_with_crews = movies_with_ratings.join(crew, "tconst").filter(F.col("directors").isNotNull())
    movies_with_crews = movies_with_crews.withColumn("decade", (F.col("startYear").cast("int") / 10).cast("int") * 10)
    movies_exploded = movies_with_crews.withColumn("director", F.explode("directors"))

    director_decade_stats = movies_exploded.groupBy("director", "decade").agg(
        F.avg("averageRating").alias("avg_rating"),
        F.count("tconst").alias("movie_count")
    )

    director_decade_count = director_decade_stats.groupBy("director") \
        .agg(F.countDistinct("decade").alias("decade_count"))

    multi_decade_directors = director_decade_count.filter(F.col("decade_count") > 1)

    director_decade_stats = director_decade_stats.join(multi_decade_directors, "director")

    windowSpec = Window.partitionBy("director").orderBy(F.desc("avg_rating"))

    director_best_decade = director_decade_stats.withColumn("rank", F.row_number().over(windowSpec)) \
        .filter(F.col("rank") == 1) \
        .drop("rank")

    director_best_decade = director_best_decade.join(name_df, director_best_decade.director == name_df.nconst, "left") \
        .select("director", "primaryName", "decade", "avg_rating", "movie_count")

    director_best_decade.show(truncate=False)
    print(f"Загальна кількість записів у відповіді: {director_best_decade.count()}")
    director_best_decade.coalesce(1).write.option("header", "true").csv(f"{RESULTS_DIR}/directors_best_decade.csv")
    return director_best_decade

 3. Які режисери, що працювали над хоча б п’ятьма фільмами в жанрі "Thriller", отримують середній рейтинг не нижче 7.0?

In [39]:
def top_thriller_directors() -> DataFrame:
    print("\nРежисери в жанрі 'Thriller', які мають мінімум 5 фільмів та середній рейтинг ≥ 7.0:")
    thriller_movies = basics.filter((F.array_contains(F.col("genres"), "Thriller")) &
                                    (F.col("startYear").isNotNull()))
    with_ratings = thriller_movies.join(ratings, "tconst")
    with_directors = with_ratings.join(crew, "tconst").filter(F.col("directors").isNotNull())
    exploded = with_directors.withColumn("director", F.explode("directors"))

    result = exploded.groupBy("director").agg(
        F.count("tconst").alias("film_count"),
        F.avg("averageRating").alias("avg_rating")
    ).filter((F.col("film_count") >= 5) & (F.col("avg_rating") >= 7.0)).orderBy(F.desc("avg_rating"))

    final = result.join(name_df, result.director == name_df.nconst, "left") \
                  .select("director", "film_count", "avg_rating", "primaryName")
    final.show(truncate=False)
    print(f"Загальна кількість записів у відповіді: {final.count()}")
    final.coalesce(1).write.option("header", "true").csv(f"{RESULTS_DIR}/top_thriller_directors.csv")
    return final



4. Які документальні фільми мають рейтинг вище 7.5?

In [40]:
def high_rated_documentaries() -> DataFrame:
    print("\nДокументальні фільми з рейтингом > 7.5:")
    documentaries = basics.filter((F.col("titleType") == "movie") &
                                  (F.col("genres").isNotNull()) &
                                  (F.array_contains(F.col("genres"), "Documentary")))
    docs_with_ratings = documentaries.join(ratings, "tconst").filter(F.col("averageRating") > 7.5)
    result = docs_with_ratings.select("tconst", "primaryTitle", "averageRating", "numVotes")
    result.orderBy(F.desc("averageRating")).show(truncate=False)
    print(f"Загальна кількість записів у відповіді: {result.count()}")
    result.coalesce(1).write.option("header", "true").csv(f"{RESULTS_DIR}/high_rated_documentaries.csv")
    return result


5. Хто з осіб працює як актори, так і режисери?

In [41]:
def dual_role_persons() -> DataFrame:
    print("\nОсоби, які працюють як актори, так і режисери:")
    actors = principals.filter(F.col("category").isin("actor", "actress")).select("nconst").distinct()
    directors = crew.filter(F.col("directors").isNotNull()) \
                    .withColumn("director", F.explode("directors")) \
                    .select(F.col("director").alias("nconst")).distinct()

    dual_role = actors.join(directors, "nconst", "inner")

    result = dual_role.join(name_df, "nconst", "left") \
                      .filter(F.col("primaryName").isNotNull()) \
                      .filter(F.col("primaryName").rlike("^[A-Za-z .'-]+$")) \
                      .select("nconst", "primaryName")

    result.orderBy("primaryName").show(truncate=False)
    print(f"Загальна кількість записів у відповіді: {result.count()}")
    result.coalesce(1).write.option("header", "true").csv(f"{RESULTS_DIR}/dual_role_persons.csv")
    return result




6. Які режисери, що працюють у не менше ніж 3 жанрах, мають стабільно високий середній рейтинг (вище 7.5)?

In [49]:
def versatile_directors() -> DataFrame:
    print("\nРежисери, що працюють у ≥3 жанрах та мають середній рейтинг >7.5:")
    games = basics.filter((F.col("titleType") == "videoGame") & (F.col("genres").isNotNull()))
    games_with_info = games.join(ratings, "tconst") \
                           .join(crew, "tconst") \
                           .filter(F.col("directors").isNotNull())

    df_exp = games_with_info.withColumn("director", F.explode("directors")) \
                             .withColumn("genre", F.explode("genres"))

    director_stats = df_exp.groupBy("director").agg(
        F.countDistinct("genre").alias("genre_count"),
        F.avg("averageRating").alias("avg_rating")
    )

    result = director_stats.filter((F.col("genre_count") >= 3) & (F.col("avg_rating") > 7.5))

    final_result = result.join(name_df, result.director == name_df.nconst, "left") \
                          .select("primaryName", "director", "genre_count", "avg_rating") \
                          .orderBy(F.desc("avg_rating"))

    result.show(truncate=False)
    print(f"Загальна кількість записів у відповіді: {result.count()}")
    result.coalesce(1).write.option("header", "true").csv(f"{RESULTS_DIR}/versatile_directors.csv")
    return result

In [50]:
versatile_directors()


Режисери, що працюють у ≥3 жанрах та мають середній рейтинг >7.5:
+----------+-----------+------------------+
|director  |genre_count|avg_rating        |
+----------+-----------+------------------+
|nm12555710|3          |8.2               |
|nm10742503|3          |7.866666666666668 |
|nm3526377 |3          |7.724999999999999 |
|nm0013911 |3          |9.1               |
|nm0693599 |3          |8.3               |
|nm0473614 |8          |7.892307692307693 |
|nm1107898 |3          |8.3               |
|nm1210786 |3          |7.7               |
|nm11683849|3          |7.900000000000001 |
|nm12516156|3          |8.2               |
|nm9465459 |3          |7.599999999999999 |
|nm9107624 |3          |7.599999999999999 |
|nm3711553 |3          |8.1               |
|nm0962779 |4          |7.5285714285714285|
|nm1659910 |3          |7.900000000000001 |
|nm2138837 |6          |7.6000000000000005|
|nm2712877 |3          |8.2               |
|nm3182289 |3          |8.3               |
|nm099386

DataFrame[director: string, genre_count: bigint, avg_rating: double]

In [46]:
english_high_rated_movies()
directors_best_decade()
top_thriller_directors()
high_rated_documentaries()
dual_role_persons()



Англомовні фільми з рейтингом > 7.0:
+---------+----------------------------------------------+---------+-------------+--------+
|tconst   |primaryTitle                                  |startYear|averageRating|numVotes|
+---------+----------------------------------------------+---------+-------------+--------+
|tt0001498|The Battle of Trafalgar                       |1911     |7.7          |19      |
|tt0002479|Seal Hunting in Newfoundland                  |1912     |7.2          |6       |
|tt0002572|Wenn die Maske fällt                          |1912     |7.4          |12      |
|tt0002637|Arizona                                       |1913     |7.9          |22      |
|tt0003150|Miraklet: Tavlor ur det katolska samfundslivet|1913     |7.1          |19      |
|tt0003386|Sodoms Ende                                   |1913     |7.7          |19      |
|tt0003748|Captain Alvarez                               |1914     |7.9          |21      |
|tt0003883|The Child of Paris             

DataFrame[nconst: string, primaryName: string]